<a href="https://colab.research.google.com/github/abisubramanya27/CS6910_Assignment2/blob/main/partB/src/Assignment2_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Mounting drive to store dataset
from google.colab import drive

drive.mount('/content/gdrive')
#!rm -rf inaturalist_12K

Mounted at /content/gdrive


In [4]:
%%time
# %cd gdrive/MyDrive/assignments/cs6910/A2/Data
# !pwd
!cp gdrive/MyDrive/assignments/cs6910/A2/Data/inaturalist_12K.zip .
#!gdown --id 11SGStqp8Vug2GDzSpJDwQYHThLIjZFQn
!unzip -q inaturalist_12K.zip
!ls

# import zipfile
# import concurrent.futures

# zf = zipfile.ZipFile('inaturalist_12K.zip')

# def unzip(file):
#     zf.extract(file)

# with concurrent.futures.ProcessPoolExecutor() as executor:
#     executor.map(unzip, zf.infolist())

gdrive	inaturalist_12K  inaturalist_12K.zip  sample_data
CPU times: user 2.32 s, sys: 405 ms, total: 2.72 s
Wall time: 8min 51s


In [ ]:
# !pip install split-folders

In [ ]:
# import splitfolders

# # Splitting the training data into training and validation set
# splitfolders.ratio('./inaturalist_12K/train', output='./inaturalist_12K/output', seed=1337, ratio=(.9, .1), group_prefix=None)

In [5]:
import os
def print_count_classes_in_valid():
  class_count_valid = {}
  for subdir, dirs, files in os.walk('./inaturalist_12K/val'):
      for file in files:
        class_count_valid[subdir] = class_count_valid.get(subdir,0)+1

  print(class_count_valid)

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, \
GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [7]:
from tensorflow.keras import regularizers

def build_model_partA(inp_img_shape, K_list, F_list, no_neurons_dense, no_classes = 10, pooling_list = ['max']*5, activation_fn_list = ['relu']*6, 
                      P_list = ['valid']*10, S_list = [1]*10, reg_list = ['none']*7, lambda_ = 0.01, BN_yes = False, dropout_p = 0):
    '''
    Function to build the model comprising (5 conv+relu+maxpooling layers + 1 dense FC layer) for part A in keras
    Arguments :
        inp_img_shape -- shape of input image
        K_list -- List of number of filters in each non FC layer
        F_list -- List of size of filters (assumed same dimension in width and height) in each non FC layer  
        no_neurons_dense -- Number of neurons in the dense FC layer
        no_classes -- Number of output classes in the classification problem
        pooling_list -- List of pooling layer option for each conv+pooling block ('max' : MaxPooling2D, 'avg': AveragePooling2D)
        activation_fn_list -- List of activation function in each convolution layer and the onne hidden FC layer
        P_list -- List of padding options in each non FC layer 
                  ('valid' : no padding, 'same' : padding to make input and output same dimensions)
        S_list -- List of strides (assumed equal in width and height) in each non FC layer
        reg_list -- List of regularization options for the convolution, one hidden FC and output layers ('none' : no regularization, 'L2' , 'L1')
        lambda_ -- weight decay hyperparameter for regularisation
        BN_yes -- True : Batch normalisation (BN) should be used, False : BN should not be used
        dropout_p -- Probability of dropping out a neuron
                     (The dropout is added for the single dense hidden layer alone after referring to many CNN architecture papers)

    Returns :
        model -- The keras sequential model of the CNN created
    '''
    get_regularization = {
        'none': None,
        'L1': regularizers.l1(lambda_),
        'L2': regularizers.l2(lambda_)
    }

    get_pooling_layer = {
        'max': MaxPooling2D,
        'avg': AveragePooling2D
    }

    model = Sequential()
    # First layer
    model.add(Conv2D(filters = K_list[0], kernel_size = (F_list[0], F_list[0]), strides = (S_list[0], S_list[0]), 
                     padding = P_list[0], input_shape = inp_img_shape, kernel_regularizer = get_regularization[reg_list[0]]))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation_fn_list[0]))
    model.add(get_pooling_layer[pooling_list[0]](pool_size=(F_list[1], F_list[1]), strides = (S_list[1], S_list[1]), padding = P_list[1]))

    # 4 Conv-relu-MaxPooling layers
    for l in range(1, 5):
        model.add(Conv2D(filters = K_list[2*l], kernel_size = (F_list[2*l], F_list[2*l]), strides = (S_list[2*l], S_list[2*l]), 
                         padding = P_list[2*l], kernel_regularizer = get_regularization[reg_list[l]]))
        if BN_yes:
            model.add(BatchNormalization())
        model.add(Activation(activation_fn_list[l]))
        model.add(get_pooling_layer[pooling_list[l]](pool_size = (F_list[2*l+1], F_list[2*l+1]), strides = (S_list[2*l+1], S_list[2*l+1]), padding = P_list[2*l+1]))
    
    # 1 dense FC layer
    model.add(Flatten())
    model.add(Dropout(dropout_p))
    model.add(Dense(units = no_neurons_dense, kernel_regularizer = get_regularization[reg_list[5]]))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation = activation_fn_list[5]))

    # Output layer
    model.add(Dense(units = no_classes, kernel_regularizer = get_regularization[reg_list[6]]))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation = 'softmax'))

    return model
    

In [8]:
!pip install --upgrade wandb
!wandb login 6746f968d95eb71e281d6c7772a0469574430408

     |████████████████████████████████| 2.1MB 4.3MB/s 
     |████████████████████████████████| 133kB 16.9MB/s 
     |████████████████████████████████| 102kB 9.6MB/s 
     |████████████████████████████████| 163kB 27.7MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=eb16feda86867369e9757bace4c4dc600e25b7c6f64dff7701b99ff584635d02
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=e6e57e23d3e503c93e30298b2fb6a0cca6c48fb9e306b0a48a5467336c98b393
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
from tensorflow.keras import layers

# Model for resizing and rescaling images
image_rescale = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255)
])

# Model for performing random transformations for data augmentation
data_augmentation = Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomTranslation(0.2, 0.2),
    layers.experimental.preprocessing.RandomZoom(0.2, 0.2),
    layers.experimental.preprocessing.RandomContrast(0.2)
])

def prepare_data(data_path, inp_img_shape, batch_size, img_preprocess, data_augmentation, data_augment_yes = False, shuffle = True):
    AUTOTUNE = tf.data.AUTOTUNE
    dataset = image_dataset_from_directory(
        data_path, labels='inferred', color_mode='rgb', batch_size=batch_size, image_size=inp_img_shape[:-1], shuffle=shuffle,
        seed=123, label_mode='categorical'
    )
    
    dataset = dataset.map(lambda x, y: (img_preprocess(x), y), num_parallel_calls=AUTOTUNE)

    # Use data augmentation only if data_augment_yes == True (Training set only requires data augmentation)
    if data_augment_yes:
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting on datasets
    return dataset.prefetch(buffer_size=AUTOTUNE)


def data_generator(train_data_path, inp_img_shape, batch_size, data_augment_yes = False, val_data_path = None, test_data_path = None):
    train_data = prepare_data(train_data_path, inp_img_shape, batch_size, image_rescale, data_augmentation, data_augment_yes)
    val_data = None
    if val_data_path is not None:
        val_data = prepare_data(val_data_path, inp_img_shape, batch_size, image_rescale, data_augmentation, False)
    test_data = None
    if test_data_path is not None:
        test_data = prepare_data(test_data_path, inp_img_shape, batch_size, image_rescale, data_augmentation, False)
    
    return train_data, val_data, test_data


In [10]:
import wandb
from wandb.keras import WandbCallback

In [11]:
def train_model(model, train_data, loss_function, optimizer = 'adam', learning_rate = 1e-3, epochs = 10, val_data = None):
    if optimizer == 'adam':
        model.compile(optimizer = Adam(learning_rate=learning_rate), loss = loss_function, metrics = ['accuracy'])
    elif optimizer == 'momentum':
        model.compile(optimizer = SGD(learning_rate=learning_rate, momentum = 0.9), loss = loss_function, metrics = ['accuracy'])
    elif optimizer == 'rmsprop':
        model.compile(optimizer = RMSprop(learning_rate=learning_rate), loss = loss_function, metrics = ['accuracy'])
    elif optimizer == 'nesterov':
        model.compile(optimizer = SGD(learning_rate=learning_rate, momentum = 0.9, nesterov = True), loss = loss_function, metrics = ['accuracy'])
    elif optimizer == 'nadam':
        model.compile(optimizer = Nadam(learning_rate=learning_rate), loss = loss_function, metrics = ['accuracy'])
    else:
        model.compile(optimizer = SGD(learning_rate=learning_rate), loss = loss_function, metrics = ['accuracy'])

    model.fit(train_data,
              epochs = epochs, 
              validation_data = val_data,
              verbose = 2,
              callbacks = [WandbCallback(monitor='val_accuracy'), EarlyStopping(monitor='val_accuracy', patience=5)])
    
    return model


In [12]:
from tensorflow.keras.models import Model
get_pretrained_function = {
    'Xception' :          (lambda : tf.keras.applications.xception.Xception(weights='imagenet', include_top=False)),
    'ResNet50' :          (lambda : tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False)),
    'InceptionV3' :       (lambda : tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False)),
    'InceptionResNetV2' : (lambda : tf.keras.applications.inception_resnet_v2.InceptionResNetV2(weights='imagenet', include_top=False))
}

get_k_value = {
    'Xception' :          116,
    'ResNet50' :          143,
    'InceptionV3' :       249,
    'InceptionResNetV2' : 617
}

In [24]:
def get_pretrained_model(model_name='InceptionResNetV2', no_neurons_dense=1024, k_value=617, no_classes=10):
  pretrained_model = get_pretrained_function[model_name]()
  # add a global spatial average pooling layer
  x = pretrained_model.output
  x = GlobalAveragePooling2D()(x)
  # let's add a fully-connected layer
  x = Dense(no_neurons_dense, activation='relu')(x)
  # and a logistic layer -- let's say we have 200 classes
  predictions = Dense(no_classes, activation='softmax')(x)
  model = Model(inputs=pretrained_model.input, outputs=predictions)
  
  for layer in pretrained_model.layers:
    layer.trainable = False

  #for i, layer in enumerate(pretrained_model.layers):
  #  print(i, layer.name)

  for layer in model.layers[:k_value]:
    layer.trainable = False
  for layer in model.layers[k_value:]:
    layer.trainable = True

  return model

In [25]:
def CNN_pretrained(model, inp_img_shape, train_data_path, config, no_classes = 10, val_data_path = None, test_data_path = None, sweep = False, 
                   wandb_log = True):
    
    id = ''
    if not sweep and wandb_log:
        id = wandb.util.generate_id()
        run = wandb.init(id = id, project="assignment2", entity="abisheks", reinit=True, config=config)
        
    tf.keras.backend.clear_session()
    
    train_data, val_data, test_data = data_generator(train_data_path, inp_img_shape, config['batch_size'], config['data_augmented'], 
                                                     val_data_path, test_data_path)
    model = train_model(model, train_data, config['loss_function'], config['optimizer'], config['learning_rate'], config['epochs'], val_data)
    
    if test_data is not None:
        test_loss, test_accuracy = model.evaluate(test_data, use_multiprocessing = True, workers = 4)
        print(f'Test Accuracy : {round(test_accuracy*100, 2)} | Test Loss : {round(test_loss, 4)}')
        if wandb_log:
            wandb.log({'test_accuracy': round(test_accuracy*100, 2), 'test_loss': round(test_loss, 4)})

    if not sweep and wandb_log:
        run.finish()

    return model, id


In [26]:
# Hyperparameters for building the model for Part-A

inp_img_shape_2 = (224, 224, 3)                                 # Shape of input image from data
no_classes_2 = 10                                               # Number of output classes in the classification problem


config_2 = {
    "learning_rate": 1e-3,                                      # Hyperparameter for updating the parameters in gradient descent
    "epochs": 10,                                               # Number of epochs to train the model   
    "optimizer": 'nesterov',                                    # Gradient descent algorithm used for the parameter updation
    "batch_size": 64,                                           # Batch size used for the optimizer
    "loss_function": 'categorical_crossentropy',                # Loss function used in the optimizer
    "no_neurons_dense": 512,                                    # Number of neurons in the dense FC layer
    "data_augmented": False,                                    # True : Data augmentation is done during training, False : No data augmentation done
    "model_name": 'InceptionResNetV2'                           # Name of the pretrained model
}


# PART-B, Question 1 -- Fine tuning a model pre-trained on Imagenet to iNaturalist dataset
# model = get_pretrained_model(config_2['model_name'], config_2['no_neurons_dense'], get_k_value[config_2['model_name']], no_classes_2)
# modelA, _ = CNN_pretrained(model, inp_img_shape_2, './inaturalist_12K/train', config_2, no_classes_2, './inaturalist_12K/val', './inaturalist_12K/test')

In [27]:
sweep_config = {
    'name': 'CNN_pretrained',
    'method': 'bayes',                   # Possible search : grid, random, bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'model_name': {
            'values': ['InceptionResNetV2', 'InceptionV3', 'ResNet50' , 'Xception']
        },
        'data_augmented': {
            'values': [True, False]
        },
        'no_neurons_dense': {
            'values': [512,1024]
        },
        'optimizer': {
            'values': ['adam', 'nesterov', 'rmsprop']
        }
    }
}

In [28]:
def sweep_wrapper(data_path = './inaturalist_12K'):
  
    # Wrapper function to call the CNN pretrained function for sweeping with different hyperparameters

    # Initialize a new wandb run
    run = wandb.init(config=config_2, reinit=True)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    #print(config)

    wandb.run.name = f'mn_{config.model_name}_nd_{config.no_neurons_dense}_op_{config.optimizer}'
    wandb.run.name += '_da' if config.data_augmented else '' 
    wandb.run.save()
    print(wandb.run.name)
    model = get_pretrained_model(config.model_name, config.no_neurons_dense, get_k_value[config.model_name], no_classes_2)
    # Sweep uses L2 regularisation as default as given in the question
    modelA, _ = CNN_pretrained(model, inp_img_shape_2, f'{data_path}/train', config_2, no_classes_2, f'{data_path}/val', sweep = True, 
                               wandb_log = True)
    run.finish()

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="abisheks", project="assignment2")
# sweep_id = "abisheks/assignment2/nhoqsu5c"
wandb.agent(sweep_id, lambda : sweep_wrapper())

Create sweep with ID: nhoqsu5c
Sweep URL: https://wandb.ai/abisheks/assignment2/sweeps/nhoqsu5c


wandb: Agent Starting Run: 1flth26y with config:
wandb: 	data_augmented: True
wandb: 	model_name: ResNet50
wandb: 	no_neurons_dense: 512
wandb: 	optimizer: adam


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


mn_ResNet50_nd_512_op_adam_da
94773248/94765736 [==============================] - 1s 0us/step
Found 9006 files belonging to 10 classes.
Found 1004 files belonging to 10 classes.
Epoch 1/10
141/141 - 158s - loss: 2.1510 - accuracy: 0.2173 - val_loss: 2.2946 - val_accuracy: 0.1056
Epoch 2/10
141/141 - 93s - loss: 1.9969 - accuracy: 0.2898 - val_loss: 2.1803 - val_accuracy: 0.1932
Epoch 3/10
141/141 - 92s - loss: 1.9122 - accuracy: 0.3297 - val_loss: 2.0228 - val_accuracy: 0.2898
Epoch 4/10
